# Westeros tutorial - Adding DACCS in climate mitigation scenario (v-CO2)
In the previous tutorials, we have learnt how to create a baseline scenario (`westeros_baseline.ipynb`) and add emissions bounds (`westeros_emissions_bounds.ipynb`) to the baseline scenario. Here, we will show how to include an additional/new technology to a MESSAGE model. While the combination of currently existing technologies might be able to deliver the Paris targets, the deployment of some new technologies might improve the probability of meeting the targets and/or reducing the costs. These technologies include CO2 removal (CDR) technologies. Hence, in this tutorial, we will use direct air carbon capture and storage (DACCS) as an example of new technologies to be considered in climate mitigation pathways. 

In order to smoothly follow this tutorial, you have to alrady have the MESSAGEix framework installed and working. Additionally, you should have run the Westeros baseline and emissions bounds scenarios successfully as this tutorial is built on top of those scenarios.

If all set, we can start by importing all the packages we need and connect to a database that store the scenario input and results. We can also name the model as `Westeros Electrified` here.

In this tutorial, we will use add_dac tool which requires user to specify the location of the data, in yaml format. As such, we use os package to help us specifying the yaml file.

In [1]:
import os

import pandas as pd
import ixmp
import message_ix
import warnings
warnings.filterwarnings('ignore')

from message_ix.utils import make_df
from message_ix.tools.add_dac import *



%matplotlib inline

mp = ixmp.Platform()

model = "Westeros Electrified"

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


After we are connected to the database, we can call the prevously run `"emission_bound"` scenario as our base model and clone the data before we start adding DACCS to the model. As prevoiusly mentioned, to run this tutorial, you have to have succesfully run the `"emission_bound"` scenario, which was built based on the `"baseline"` scenario.

In [18]:
base = message_ix.Scenario(mp, model=model, scenario="multinode_hub")

scenario = base.clone(
    model,
    "multinode_hub_emission_bound",
    "multinode_hub with emission bound",
    keep_solution=False,)
scenario.check_out()

year_df = scenario.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scenario.set("year")


In [22]:
# add "World" node
scenario.add_set("node","World")

# add "WorldEmiss" technology
scenario.add_set("technology","WorldEmiss")

# add "CO2" commodity
scenario.add_set("commodity","CO2")


In [20]:
# add emission factor
# First we introduce the emission of CO2 and the emission category GHG
scenario.add_set("emission", "CO2")
scenario.add_cat("emission", "GHG", "CO2")

# Then we add new units to the model library (needed only once)
mp.add_unit("tCO2/kWa")
mp.add_unit("MtCO2")

model_years = list(set(act_years)) 

# Last we add CO2 emissions to the WorldEmiss
emission_factor = make_df(
    "emission_factor",
    node_loc="World",
    year_vtg=model_years,
    year_act=model_years,
    mode="standard",
    unit="tCO2/kWa",
    technology="WorldEmiss",
    emission="CO2",
    value=1,
)
scenario.add_par("emission_factor", emission_factor)
scenario.par("emission_factor")

,node_loc,technology,year_vtg,year_act,mode,emission,value,unit
0,World,WorldEmiss,720,720,standard,CO2,1.0,tCO2/kWa
1,World,WorldEmiss,700,700,standard,CO2,1.0,tCO2/kWa
2,World,WorldEmiss,710,710,standard,CO2,1.0,tCO2/kWa


In [23]:
# adding emission output
# Parametrization of "output" for import technologies
# The destination of import is the level of "secondary" in each country
base_output = {
    "technology": "WorldEmiss",
    "commodity": "CO2",
    "level": "secondary",
    "year_vtg": model_years,
    "year_act": model_years,
    "mode": "standard",
    "time": "year",
    "time_dest": "year",
    "value": 1,
    "unit": "-",
}

# We add this data for each node (other than "hub")
out = make_df("output", **base_output, node_loc="World", node_dest="World")
scenario.add_par("output", out)

In [ ]:
# add relation, relating coal activity with WorldEmiss activity
# don't forget to include relation upper and lower to 0

In [ ]:
# add emission bound
scenario.add_par(
    "bound_emission", ["World", "GHG", "all", "cumulative"], value=1500.0, unit="MtCO2"
)

**Solve scenario without DACCS**

In [ ]:
scenario.commit(comment="Multinode scenario emission bound without daccs")
scenario.set_as_default()

scenario.solve()
scenario.var("OBJ")["lvl"]

print('Objective value: ', scenario.var("OBJ")["lvl"])

# Adding DACCS description
First step of adding DACCS as a technology in the model is by including DACCS into the `"technology"` set.

In [ ]:
mp.add_unit("USD/(tCO2/yr)")
mp.add_unit("USD/(tCO2/yr)/yr")
mp.add_unit("USD/tCO2")
mp.add_unit("tCO2/tCO2")
mp.add_unit("tCO2")
mp.add_unit("Mt CO2/yr")


filepath = os.path.join(os.getcwd(), "data/tech_data.yaml")
add_dac(scenario, filepath=filepath)


Similar to what we did when generating the `"baseline"` scenario, the first thing we need to do is defining the input and output comodities of each technology. 

# Solve Statement
Finally, this is the solve statement

In [ ]:
scenario.commit(comment="Adding daccs using add_dac tool")
scenario.set_as_default()

scenario.solve()
scenario.var("OBJ")["lvl"]

print('Objective value: ', scenario.var("OBJ")["lvl"])

# Plotting Results and Compare
Finally, this is the plotting results command to compare emissions bound scenarios with and without DACCS

In [ ]:
# Create a Reporter object to describe and carry out reporting
# calculations and operations (like plotting) based on `scenario`
# Add keys like "plot activity" to describe reporting operations.
# See tutorial/utils/plotting.py
from message_ix.reporting import Reporter
from message_ix.util.tutorial import prepare_plots

rep_ori = Reporter.from_scenario(base)
rep_new = Reporter.from_scenario(scenario)

## System acticity

In [ ]:
print("Without DACCS")
prepare_plots(rep_ori)
rep_ori.set_filters(t=["coal_ppl", "wind_ppl"])
rep_ori.get("plot activity")
plt.show()

print("With DACCS")
prepare_plots(rep_new)
rep_new.set_filters(t=["coal_ppl", "wind_ppl"])
rep_new.get("plot activity")
plt.show()

### DACCS Capacity

In [ ]:
prepare_plots(rep_new)
rep_new.set_filters(t=["daccs"])
rep_new.get("plot daccs capacity")
plt.show()

## Emissions

In [ ]:
print("Without DACCS")
prepare_plots(rep_ori)
rep_ori.set_filters(t=["coal_ppl", "wind_ppl"])
rep_ori.get("plot emission")
plt.show()

print("With DACCS")
prepare_plots(rep_new)
rep_new.set_filters(t=["coal_ppl", "wind_ppl","daccs"])
rep_new.get("plot emission")
plt.show()

## Close the connection with the database

In [ ]:
mp.close_db()